# Dark Souls II Reviews (2025)

In [ ]:
from get_data import *

In [ ]:
reviews = get_data()

## Topic Modeling:
- Exploring certain aspects on why people like the game
    - Also get critiques of the game in positive reviews (if any but there sure is considering DS2's reputation in the community)

- Exploring why people don't like the game:
    - Also get positive aspects within this subset of the reviews

Splitting the reviews by how many do and don't recommend buying the game:

In [17]:
pos_reviews = reviews[reviews['voted_up'] == True]
neg_reviews = reviews[reviews['voted_up'] == False]

In [18]:
pos_reviews.shape, neg_reviews.shape

((37387, 9), (8176, 9))

### Top2Vec:

In [19]:
from top2vec import Top2Vec

In [20]:
docs = reviews['review'].tolist()
pos_docs = pos_reviews['review'].tolist()
neg_docs = neg_reviews['review'].tolist()

In [ ]:
def topics(documents):
    mdl = Top2Vec(
        documents=documents,
        contextual_top2vec=True,
        speed="fast-learn",
        workers=8,
    )
    return mdl

In [ ]:
mdl = topics(docs)

2025-04-02 18:10:04,112 - top2vec - INFO - Pre-processing documents for training


c:\Users\bdion\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2025-04-02 18:10:12,279 - top2vec - INFO - Creating vocabulary embedding
Embedding vocabulary:  14%|█▍        | 12/87 [42:00<7:29:34, 359.66s/it] 

In [ ]:
pos_mdl = topics(pos_docs)

In [ ]:
neg_mdl = topics(neg_docs)